# Problem 10.3

## Part j

In [1]:
set more off
input z Y0M0 Y1M0 Y0M1 Y1M1 M0 M1
0 0 0 0 0 0 0 
0 0 0 0 0 0 1
0 0 0 0 0 1 1
0 0 1 0 1 0 0
0 0 1 0 1 0 1 
0 0 1 0 1 1 1
1 1 0 1 1 0 0
1 1 0 1 1 0 1
1 1 0 1 1 1 1
1 0 1 1 1 0 0
1 0 1 1 1 0 1
1 0 1 1 1 1 1
end




             z       Y0M0       Y1M0       Y0M1       Y1M1         M0         M1


In [2]:
tabstat Y0M0 Y1M0 Y0M1 Y1M1, stat(mean)	

gen M = .
gen Y = .



   stats |      Y0M0      Y1M0      Y0M1      Y1M1
---------+----------------------------------------
    mean |       .25        .5        .5       .75
--------------------------------------------------

(12 missing values generated)

(12 missing values generated)


In [3]:
//coefmat
capture program drop coef
program define coef, rclass
	replace M = M0*(1-z) + M1*z
	replace Y = Y0M0*(1-z)*(1-M) + Y1M0*(z)*(1-M) + Y0M1*(1-z)*(M) + Y1M1*(z)*(M)	
	qui reg Y M z
	return scalar coy = _b[_cons]
	return scalar com = _b[M]
	return scalar coz = _b[z]
	return scalar nocoli = _se[z]
end

qui tsrtest z r(coy) using co_y.dta, overwrite: coef 
qui tsrtest z r(com) using co_m.dta, overwrite: coef
qui tsrtest z r(coz) using co_z.dta, overwrite: coef 
qui tsrtest z r(nocoli) using nocoli.dta, overwrite: coef 

In [4]:
// tcoefmat
capture program drop tcoef
program define tcoef, rclass
	replace M = M0*(1-z) + M1*z
	replace Y = Y0M0*(1-z)*(1-M) + Y1M0*(z)*(1-M) + Y0M1*(1-z)*(M) + Y1M1*(z)*(M)	
	qui reg Y z
	return scalar tcoy = _b[_cons]
	return scalar tcoz = _b[z]
end

qui tsrtest z r(tcoy) using tco_y.dta, overwrite: tcoef
qui tsrtest z r(tcoz) using tco_z.dta, overwrite: tcoef 

In [5]:
// mcoefmat
capture program drop mcoef
program define mcoef, rclass
	replace M = M0*(1-z) + M1*z
	qui reg M z
	return scalar mcom = _b[_cons]
	return scalar mcoz = _b[z]
end

qui tsrtest z r(mcom) using mco_m.dta, overwrite: mcoef 
qui tsrtest z r(mcoz) using mco_z.dta, overwrite: mcoef

In [6]:

// colMeans(na.omit(coefmat))
preserve	
qui use "co_y.dta", clear
qui rename theta co_y

qui merge 1:1 _n using "co_m.dta"
qui rename theta co_m 

qui drop _merge
qui merge 1:1 _n using "co_z.dta"
qui rename theta co_z 
qui drop _merge

//check colinearity
qui merge 1:1 _n using "nocoli.dta"
qui rename theta nocoli
qui drop _merge


qui drop if _n == 1 
// omit instances of perfect colinearity between M and Z
qui drop if nocoli==0

tabstat co_y co_m co_z,stat(mean)

restore

















   stats |      co_y      co_m      co_z
---------+------------------------------
    mean |       .25       .25       .25
----------------------------------------



In [8]:
// colMeans(na.omit(tcoefmat)))
preserve
qui use "tco_y.dta", clear
qui rename theta tco_y

qui merge 1:1 _n using "tco_z.dta"
qui rename theta tco_z 
qui drop _merge

qui /*check coli*/
qui merge 1:1 _n using "nocoli.dta"
qui rename theta nocoli
qui drop _merge

// drop the observation statistics
qui drop if _n == 1 

tabstat tco_y tco_z,stat(mean)
restore














   stats |     tco_y     tco_z
---------+--------------------
    mean |  .3333333  .3333333
------------------------------



In [9]:
//colMeans(na.omit(mcoefmat)))
preserve	
qui use "mco_m.dta", clear
qui rename theta mco_m

qui merge 1:1 _n using "mco_z.dta"
qui rename theta mco_z 
qui drop _merge
 
// drop the observation statistics
qui drop if _n == 1 
tabstat mco_m mco_z,stat(mean)
restore










   stats |     mco_m     mco_z
---------+--------------------
    mean |  .3333333  .3333333
------------------------------



## Part k

In [10]:
preserve	 

qui use "mco_z.dta", clear
qui rename theta mco_z 

qui merge 1:1 _n using "co_z.dta"
qui rename theta co_z 
qui drop _merge 

// check colinearity
qui merge 1:1 _n using "nocoli.dta"
qui rename theta nocoli
qui drop _merge

qui gen asbs = mco_z*co_z
qui drop if _n == 1 
// omit instances of perfect colinearity between M and Z
qui drop if nocoli==0


tabstat asbs,stat(mean)
restore















    variable |      mean
-------------+----------
        asbs |   .082244
------------------------

